# Climate Analysis and Exploration

### Setup

In [3]:
# Dependencies
import matplotlib
matplotlib.use('nbagg')
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [6]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [7]:
# reflect an exisiting database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

NameError: name 'engine' is not defined

In [8]:
# We can view all of the classes that automap found
Base.classes.keys()

[]

In [ ]:
# Save references to each table
Measurement = Base.classes.measurements
Station = Base.classes.station

In [ ]:
# Create our session(link) from Python to the DB
session = Session(engine)

## Exploratory Climate Analysis

In [3]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
# Calculate the date 1 year ago from today
prev_year = dt.date.today() - dt.timedelta(days=365)

In [9]:
# Perform a query to retrieve the data and precipitation scores
results = session.query(Measurement.date, 
Measurement.prcp).filter(Measurements.date >= prev_year).all()

NameError: name 'session' is not defined

In [5]:
# Save the query results as a Pandas DataFrame and set the index to the data column
df = pd.DataFrame(results, columns=['date','precipitation'])
df.set_index(df['date'], inplace=True)

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
df.plot(x_compat=True)

In [ ]:
# Rotate the xticks for the dates
plt.xticks(rotation='45')
plt.tight_layout()
plt.show

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
df.describe()

In [10]:
# How many stations are available in this dataset?
session.query(func.count(Station.station)).all()

NameError: name 'session' is not defined

In [ ]:
# What are the most active stations?
# List the stations and the counts in descending order?
session.query(Measurement.station, func.count(Measurement.station)).\
group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()

In [11]:
# Using the station id from the previous query, calculate the lowest temperature recorded,
# highest temperature recorded, and average temperature most active station?
session.query(func.min(Measurement.tobs),
func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
    filter(Measurement.station == 'USCO00519281').all()

NameError: name 'session' is not defined

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
import datetime as dt
from pandas.plotting import table
prev_year = dt.date.today() - dt.timedelta(days=365)

results = session.query(Measurement.tobs).\
    filter(Measurements.station == '').\
    filter(Measurements.data >= prev_year).all()
df = pd.DataFrame(results, columns=['tobs'])
df.plot.hist(bins=12)
plt.tight_layout()
plt.show()

In [ ]:
# Write a function called 'calc_temps' that will accept start date and end date ni the format '%Y-%m-%d'
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format 
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    return session.query(func.min(Measurement.tobs),
func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
    filter(Measurement.date >=
start_date).filter(Measurement.date <= end_date).all()
print(calc_temps('',''))

In [12]:
# Use your previous function 'calc_temps' to calculate the tmin, tavg, and tmax
# for your trip using the previous year's data for those same dates.
import datetime as dt

prev_year_start = dt.date(2018, 1, 1) - dt.timedelta(days=365)
prev_year_end = dt.date(2018, 1, 7) - dt.timedelta(days=365)

tmin, tavg, tmax = calc_temps(prev_year_start.strfttime("%Y-%m-%d"),
prev_year_end.strftime("%Y-%m-%d"))[0]
print(tmin, tavg, tmax)

NameError: name 'calc_temps' is not defined

In [ ]:
# PLot the results from your previous query as a bar chart
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
fig, ax = plt.subplots(figsize=plt.figaspect(2.))
xpos = 1
yerr = tmax - tmin

bar = ax.bar(xpos, tmax, yerr=yerr, alpha=0.5, color='coral',
align="center")
ax.set(xticks=range(xpos), xticklabels="a", title="Trip Avg. Temp", ylabel="Temp (F)")
ax.margins(.2,.2)
# fig.autoformat_xdate()
fig.tight_layout()
fig.show()

In [ ]:
# Calculate the rainfall per weather station for your trip dates using the previous year's matching dates
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation
start_date = '2012-01-01'
end_date = '2012-01-07'

sel = [Station.station, Station.name, Station.latitude, Station.longtitude, Station.elevation,func.sum(Measurement.prcp)]

results = session.query(*sel).\
    filter(Measurement.station == Station.station).\
    filter(Measurement.date >= start_date).\
    filter(MEasurement.date <= end_date).\
    
group_by(Station.name).order_by(func.sum(Measurement.prcp).desc()).all()
print(results)

In [ ]:
# Create a query that will calculate the daily normals
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
    
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    """ 
    
    sel = [func.min(Measurements.tobs),func.avg(Measurement.tobs),func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime()).all()

    daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called 'normals'

# Set the start and end date of the trip
trip_start = ''
trip_end = ''

# Use the start and end date to create a range of dates
trip_dates = pd.date_range(trip_start, trip_end, freq='D')

# Stip off the year and save a list of %m-%d strings
trip_month_day = trip_dates.strftime('%m-%d')

# Loop through the list of %m-%d strings and calculate the normals for each date
normals = []
for date in trip_month_day:
    normals.append(*daily_normals(date))
normals

In [ ]:
# Load the previous query results into a Pandas DataFrame and add the 'trip_dates' range as the 'date' index
df = pd.DataFrame(normals, columns=['tmin','tavg','tmax'])
df['date'] = trip_dates
df.set_index(['date'], inplace=True)
df.head()

In [ ]:
# Plot the daily normals as an area plot with stacked=False
df.plot(kind='area', stacked=False, x_compat=True, alpha=.2)
plt.tight_layout()
plt.show()